In [33]:
import requests
from bs4 import BeautifulSoup
import re
from bs4.element import Tag
import csv
from tqdm import tqdm

In [34]:
from urllib.parse import urlparse, parse_qs

# Definir opciones para iterar
tipos_publicacion = ["venta"]
tipos_propiedad = ["casa"]
regiones = ["valparaiso"]
otras = ["tarapaca", "Antofagasta", "Atacama", "Coquimbo", "Valparaíso", "Metropolitana", "bernardo-ohiggins", "Maule", "Ñuble", "Biobío", "la-Araucania", "de-los-rios", "Los-Lagos", "Aysén", "Magallanes"]

# Crear lista con todas las combinaciones posibles de URL
urls = []
for tipo_pub in tipos_publicacion:
    for tipo_prop in tipos_propiedad:
        for region in regiones:
            path = f"/{tipo_pub}/{tipo_prop}/{region}"
            url = "https://www.portalinmobiliario.com" + path
            urls.append(url)

# Imprimir las URLs generadas
for url in urls:
    print(url)


https://www.portalinmobiliario.com/venta/casa/valparaiso


In [35]:
for url in urls:
  # Send a request to the website
  response = requests.get(url)

In [36]:
for url in urls:
  # Parse the HTML content using BeautifulSoup
  soup = BeautifulSoup(response.content, "html.parser")

  # Find all the links to individual property pages
  property_links = soup.find_all("a", class_="ui-search-result__content-wrapper")

In [37]:
# Crear una lista vacía para almacenar los datos de las propiedades
property_data = []

for region_url in tqdm(urls, desc="Progress"):
    # Send a request to the region page
    region_response = requests.get(region_url)
    region_soup = BeautifulSoup(region_response.content, "html.parser")

    # Get the total number of pages for the region
    num_pages_string = region_soup.find("li", class_="andes-pagination__page-count").text
    num_pages = int(re.search(r'\d+', num_pages_string).group())

    # Loop through each page in the region
    for page_num in tqdm(range(1, num_pages + 1), desc="Pages"):
        page_url = region_url.replace("_NoIndex_True", f"_Desde_{(page_num-1)*50}_NoIndex_True")
        page_response = requests.get(page_url)
        page_soup = BeautifulSoup(page_response.content, "html.parser")

        # Find all the property links on the page
        property_links = page_soup.find_all("a", class_="ui-search-result__content-wrapper")

        # Loop through the property links and extract the information from each page
        for link in property_links:
            # Get the URL of the property page
            property_url = link.get("href")

            # Send a request to the property page
            property_response = requests.get(property_url, encoding="utf-8")
            # Parse the HTML content of the property page
            property_soup = BeautifulSoup(property_response.content, "html.parser")

            # Extract the desired information from the property page
            if isinstance(property_soup.find("h1", class_="ui-pdp-title"), Tag) & isinstance(property_soup.find("span", class_="andes-money-amount__fraction"), Tag) & isinstance(property_soup.find("div", class_="ui-vip-location"), Tag) & isinstance(property_soup.find("p", class_="ui-pdp-description__content"), Tag):
                property_title = property_soup.find("h1", class_="ui-pdp-title").text
                property_price = property_soup.find("span", class_="andes-money-amount__fraction").text
                property_location = property_soup.find("div", class_="ui-vip-location").text
                #property_description = property_soup.find("p", class_="ui-pdp-description__content").text
                # Append the property data to the list
                property_data.append({
                    "URL": property_url,
                    "Title": property_title,
                    "Price": property_price,
                    "Location": property_location,
                    #"Description": property_description
                })

# Guardar los datos en un archivo CSV
with open("property_data.csv", mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["URL", "Title", "Price", "Location"])
    writer.writeheader()
    for row in property_data:
        writer.writerow(row)

Progress: 100%|██████████| 1/1 [25:28<00:00, 1528.96s/it]
